In [2]:
import codecs
from collections import Counter, defaultdict
import random
from itertools import izip, combinations
import re
import os

## Creating the CSV file from original word, Farasa and ARZ segmenter

In [2]:
#https://docs.python.org/2/library/csv.html

import csv, codecs, cStringIO

class UTF8Recoder:
    """
    Iterator that reads an encoded stream and reencodes the input to UTF-8
    """
    def __init__(self, f, encoding):
        self.reader = codecs.getreader(encoding)(f)

    def __iter__(self):
        return self

    def next(self):
        return self.reader.next().encode("utf-8")

class UnicodeReader:
    """
    A CSV reader which will iterate over lines in the CSV file "f",
    which is encoded in the given encoding.
    """

    def __init__(self, f, dialect=csv.excel, encoding="utf-8", **kwds):
        f = UTF8Recoder(f, encoding)
        self.reader = csv.reader(f, dialect=dialect, **kwds)

    def next(self):
        row = self.reader.next()
        return [unicode(s, "utf-8") for s in row]

    def __iter__(self):
        return self

class UnicodeWriter:
    """
    A CSV writer which will write rows to CSV file "f",
    which is encoded in the given encoding.
    """

    def __init__(self, f, dialect=csv.excel, encoding="utf-8", **kwds):
        # Redirect output to a queue
        self.queue = cStringIO.StringIO()
        self.writer = csv.writer(self.queue, dialect=dialect, **kwds)
        self.stream = f
        self.encoder = codecs.getincrementalencoder(encoding)()

    def writerow(self, row):
        self.writer.writerow([s.encode("utf-8") for s in row])
        # Fetch UTF-8 output from the queue ...
        data = self.queue.getvalue()
        data = data.decode("utf-8")
        # ... and reencode it into the target encoding
        data = self.encoder.encode(data)
        # write to the target stream
        self.stream.write(data)
        # empty queue
        self.queue.truncate(0)

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

In [8]:
segmentations = dict()
all_tokens = list()
with open('new_data/Dialects Tokens Annotation - EGY.csv', mode='rb') as csvf:
    reader = UnicodeReader(csvf, delimiter=',')#, quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i, x in enumerate(reader):
        
        if x[7] == '#':
            hashtag = list()
            hash_flag = True
            hashtag.append(x[7])
            
        elif hash_flag and x[7] == '_':
            hashtag.append(x[7])
        elif hash_flag and x[7] != '_':
            hash_flag = False
            hashtag.append(x[7])
            del hashtag
            
            
            
            
        if i==0:
            print x[7]
#         if x[4].strip():
#             segmentations[x[4]] = x[7]
            
    else:
#         segmentations[u'8.'] = u'8 .'
#         segmentations[u','] = u','
#         segmentations[u'الجنَّة'] = u'ال+جن+ة'
#         segmentations[u'اممممممممم'] = u'اممممممممم'
#         segmentations[u'مليار'] = u'مليار'
#         segmentations[u'القمح'] = u'ال+قمح'
#         segmentations[u'المستوردين'] = u'ال+مستورد+ين'
#         segmentations[u'9'] = u'9'
#         segmentations[u'3'] = u'3'
#         segmentations[u'مابين'] = u'مابين'
#         segmentations[u'ولي'] = u'ولي'
#         segmentations[u'وارباب'] = u'و+ارباب'
#         segmentations[u'العمل'] = u'ال+عمل'
#         segmentations[u'كايتقام'] = u'كا+يتقام'
#         segmentations[u'كايستافدو'] = u'كا+يستافد+و'
#         segmentations[u'تستقر'] = u'تستقر'
#         segmentations[u'المقاصة'] = u'ال+مقاص+ة'
#         segmentations[u'لصندوق'] = u'ل+صندوق'
#         segmentations[u'الفلاحية'] = u'ال+فلاحي+ة'
#         segmentations[u'السنة'] = u'ال+سن+ة'
#         segmentations[u'\u0627\u0644\u062f\u0648\u0644'] = u'\u0627\u0644\u062f\u0648\u0644'
#         segmentations[u'\u0648\u062a\u0633\u0627\u0641\u0631'] = u'\u0648\u062a\u0633\u0627\u0641\u0631'
#         segmentations[u'\u0627\u0644\u0642\u0645\u0629'] = u'\u0627\u0644\u0642\u0645\u0629'
#         segmentations[u'\u0639\u0627\u0644\u064a\u0629'] = u'\u0639\u0627\u0644\u064a\u0629'
#         segmentations[u'\u0627\u062d\u0633\u0627\u0633\u0643'] = u'\u0627\u062d\u0633\u0627\u0633\u0643'
#         segmentations[u'\u0634\u062a\u0642\u062a\u0644\u0647\u0627'] = u'\u0634\u062a\u0642\u062a\u0644\u0647\u0627'
        print i+1, len(segmentations)

ليه
7077 3479


In [6]:
pAllDiacritics = re.compile(u"[\u0640\u064b\u064c\u064d\u064e\u064f\u0650\u0651\u0652\u0670]", re.U)
pAllNonCharacters = re.compile(u"[\u0020\u2000-\u200F\u2028-\u202F\u205F-\u206F\uFEFF]+", re.U)
emailRegex = re.compile("[a-zA-Z0-9\\-\\._]+@[a-zA-Z0-9\\-\\._]+$", re.U)
ALLDelimiters = u"\u0020\u0000-\u002F\u003A-\u0040\u007B-\u00BB\u005B-\u005D\u005F-\u0060\\^\u0600-\u060C\u06D4-\u06ED\ufeff"
pAllDelimiters = re.compile(u"[" + ALLDelimiters + "]+", re.U)
AllArabicLettersAndHindiDigits = u"\u0621\u0622\u0623\u0624\u0625\u0626\u0627\u0628\u0629\u062A\u062B\u062C\u062D\u062E\u062F" \
                                 + u"\u0630\u0631\u0632\u0633\u0634\u0635\u0636\u0637\u0638\u0639\u063A\u0641\u0642\u0643\u0644\u0645\u0646\u0647\u0648\u0649\u064A\u0660\u0661\u0662\u0663\u0664\u0665\u0666\u0667\u0668\u0669"
AllDigits = u"0123456789"   
AllArabicDiacritics = u"\u064B\u064C\u064D\u064E\u064F\u0650\u0651\u0652"
# ALL Arabic letters \U0621-\U063A\U0641-\U064A
AllArabicLetters = u"\u0621\u0622\u0623\u0624\u0625\u0626\u0627\u0628\u0629\u062A\u062B\u062C\u062D\u062E\u062F" \
                   + u"\u0630\u0631\u0632\u0633\u0634\u0635\u0636\u0637\u0638\u0639\u063A\u0641\u0642\u0643\u0644\u0645\u0646\u0647\u0648\u0649\u064A"

def removeNonCharacters(s):
    return pAllNonCharacters.sub(' ', s)

def removeDiacritics(s):
    return pAllDiacritics.sub('', s)



def charBasedTonkenizer(s):
    sFinal = ""
    for i, ch in enumerate(s):
        if pAllDelimiters.match(s[i:i + 1]) and s[i:i + 1] != "." and s[i:i + 1] != "," and s[i:i + 1] != "." and s[
                                                                                                                  i:i + 1] != "%":
            sFinal += " " + s[i: i + 1] + " "
        elif s[i: i + 1] == "%":
            if i > 0 and s[i:i + 1] in AllDigits:
                sFinal += s[i:i + 1] + " "
            else:
                sFinal += " " + s[i:i + 1] + " "
        elif s[i:i + 1] == "." or s[i:i + 1] == "," or s[i:i + 1] == ".":
            if 0 < i < len(s) - 1 and s[i - 1: i] in AllDigits and s[i + 1: i + 2] in AllDigits:
                sFinal += s[i: i + 1]
            elif i > 0 and s[i: i + 1] == "." and s[i - 1: i] == ".":
                sFinal += s[i: i + 1]
            elif i == 0:
                sFinal += s[i: i + 1] + " "
            elif i == len(s) - 1:
                sFinal += " " + s[i: i + 1]
            elif s[i - 1:i] in AllDigits and s[i + 1:i + 2] in AllDigits:
                sFinal += s[i: i + 1]
            else:
                sFinal += " " + s[i: i + 1] + " "
        elif s[i:i + 1] not in AllArabicLettersAndHindiDigits + u"\u0640\u064b\u064c\u064d\u064e\u064f\u0650\u0651\u0652\u0670" + u"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789ÀÁÂÃÄÅÆÇÈÉËÌÍÎÏÐÑÒÓÔÕÖØÙÚÛÜÝÞßàáâãäåæçèéêëìíîïðñòóôõöøùúûüýþÿ":
            sFinal += " " + s[i:i+1] + " "
        else:
            if (i == 0):
                sFinal += s[i: i + 1]
            else:
                if (s[i:i + 1] in AllDigits and s[i - 1: i] in (AllArabicLetters + AllArabicDiacritics)) or (
                                s[i - 1:i] in AllDigits and s[i:i + 1] in (AllArabicLetters + AllArabicDiacritics)):
                    sFinal += " " + s[i: i + 1]
                else:
                    sFinal += s[i: i + 1]

    return sFinal.strip()


def tokenize(text):
    s = removeNonCharacters(text)
    s = removeDiacritics(s)

    tnr_pattern = re.compile(u'[\t\n\r]')
    s = tnr_pattern.sub(' ', s)
    output = list()
    words = s.split()

    for i, word in enumerate(words):
        if word.startswith((u"#", u"@", u":", u";", u"http")) or emailRegex.match(word):
            output.append(word)
        else:
            for ss in charBasedTonkenizer(word).split():
                if len(ss.strip()) > 0:
                    if ss.startswith(u"\u0644\u0644"):
                        output.append(u"\u0644\u0627\u0644" + ss[2:])
                    else:
                        if len(ss.strip()) > 0:
                            output.append(ss)
    return output

In [7]:
p = re.compile(ur'#\s[\w\+]+(\s_\s[\w\+]+)*', re.U)
def remove_spaces(match):
    return match.group().replace(' ', '').replace('+', '')

In [5]:
# Set the directory you want to start from
rootDir = '../../qcri/egy_seg/DATA_SEG_All_27.03.2017/'
for dirName, subdirList, fileList in os.walk(rootDir):
    print('Found directory: %s' % dirName)
    for fname in fileList:
        if fname.endswith('.trg') and os.path.basename(dirName)== 'splits' and os.path.basename(os.path.dirname(dirName))!= 'egy_seg':
            all_tokens = list()
            with codecs.open(os.path.join(dirName,fname), encoding='utf8') as rfobj:
                
                for line in rfobj:
                    if line.strip() != "<EOTWEET>":
                        all_tokens.append(line.strip())
                    else:
                        all_tokens.append('EOTWEET')
                else:
                    dial_dir = os.path.basename(os.path.dirname(dirName))
                    with codecs.open(os.path.join('./new_data/',dial_dir,fname[:-3]+'svm'), encoding='utf8', mode='w') as rfobj:
                        single_line = ' '.join(all_tokens)
                        if os.path.basename(os.path.dirname(dirName))== 'egy_seg':
                            single_line = p.sub(remove_spaces, single_line)
                        tokenized = ' '.join(tokenize(single_line))
                        for tk in tokenized.replace(" + ","+").split():                            
                            if os.path.basename(os.path.dirname(dirName)) != 'egy_seg':                                
                                rfobj.write(tk.replace('+', ''))
                                rfobj.write('\t')
                            rfobj.write(tk)
                            rfobj.write('\n')
                        
                    
                    print len(all_tokens),
            print('\t%s' % fname)

Found directory: ../../qcri/egy_seg/DATA_SEG_All_27.03.2017/
Found directory: ../../qcri/egy_seg/DATA_SEG_All_27.03.2017/lev_seg
Found directory: ../../qcri/egy_seg/DATA_SEG_All_27.03.2017/lev_seg/splits
4986 	lev_trainfold_04.trg
674 	lev_devfold_02.trg
5591 	lev_trainfold_02.trg
1308 	lev_testfold_01.trg
4991 	lev_trainfold_01.trg
674 	lev_devfold_04.trg
1451 	lev_testfold_05.trg
1338 	lev_testfold_04.trg
1494 	lev_testfold_03.trg
4897 	lev_trainfold_05.trg
674 	lev_devfold_03.trg
650 	lev_devfold_05.trg
1407 	lev_testfold_02.trg
699 	lev_devfold_01.trg
4830 	lev_trainfold_03.trg
Found directory: ../../qcri/egy_seg/DATA_SEG_All_27.03.2017/lev_seg/splits_msa
Found directory: ../../qcri/egy_seg/DATA_SEG_All_27.03.2017/DATA_SEG_All_27.03.2017
Found directory: ../../qcri/egy_seg/DATA_SEG_All_27.03.2017/magh_seg
Found directory: ../../qcri/egy_seg/DATA_SEG_All_27.03.2017/magh_seg/splits
1137 	data_5.test.trg
1102 	data_2.test.trg
1224 	data_1.test.trg
4036 	data_4.train.trg
632 	data_3.de

In [20]:
import os
 
# Set the directory you want to start from
rootDir = 'new_data/orig4desouki/'
for dirName, subdirList, fileList in os.walk(rootDir):
    print('Found directory: %s' % dirName)
    for fname in fileList:
        if fname.endswith('.trg'): #and os.path.basename(dirName)== 'splits' and os.path.basename(os.path.dirname(dirName))!= 'egy_seg':
            all_tokens = list()
            with codecs.open(os.path.join(dirName,fname), encoding='utf8') as rfobj:                
                for line in rfobj:
                    all_tokens.append(line.strip())
                else:
                    print dirName
                    print os.path.dirname(os.path.dirname(dirName))
                    dial_dir = os.path.basename(os.path.dirname(dirName))
                    with codecs.open(os.path.join(os.path.dirname(os.path.dirname(dirName)),'egy_new_data',fname[:-3]+'src'), encoding='utf8', mode='w') as rfobj:
                        single_line = ' '.join(all_tokens)
                        single_line = p.sub(remove_spaces, single_line)
                        tokenized = ' '.join(tokenize(single_line))
                        for tk in tokenized.replace(" + ","").split():                            
                            rfobj.write(tk)
                            rfobj.write('\n')
                        
                    
                    print len(all_tokens),
            print('\t%s' % fname)

Found directory: new_data/orig4desouki/
new_data/orig4desouki/
new_data
1463 	data_5.test.trg
new_data/orig4desouki/
new_data
1415 	data_2.test.trg
new_data/orig4desouki/
new_data
1421 	data_1.test.trg
new_data/orig4desouki/
new_data
4975 	data_4.train.trg
new_data/orig4desouki/
new_data
735 	data_3.dev.trg
new_data/orig4desouki/
new_data
4928 	data_2.train.trg
new_data/orig4desouki/
new_data
1380 	data_3.test.trg
new_data/orig4desouki/
new_data
734 	data_2.dev.trg
new_data/orig4desouki/
new_data
705 	data_4.dev.trg
new_data/orig4desouki/
new_data
1397 	data_4.test.trg
new_data/orig4desouki/
new_data
805 	data_5.dev.trg
new_data/orig4desouki/
new_data
579 	data_1.dev.trg
new_data/orig4desouki/
new_data
4962 	data_3.train.trg
new_data/orig4desouki/
new_data
4809 	data_5.train.trg
new_data/orig4desouki/
new_data
5077 	data_1.train.trg


## Merging Dev and train files for EGY and MAGH

In [20]:
# rootDir = 'final_splits_all_data/joint/joint/'
rootDir = 'final_splits_all_data/joint/joint'
for dirName, subdirList, fileList in os.walk(rootDir):
    print('Found directory: %s' % dirName)
#     print os.path.dirname(dirName)
#     print os.path.basename(os.path.dirname(dirName))
#     print os.path.basename(dirName)
    for fname in fileList:
        if os.path.basename(dirName) == 'joint' and fname.startswith('magh_') and 'train' in fname: #and os.path.basename(dirName)== 'splits' and os.path.basename(os.path.dirname(dirName))!= 'egy_seg':
            print fname
            with codecs.open(os.path.join(dirName,fname[:-4]), encoding='utf8', mode='w') as rfobj:
                with codecs.open(os.path.join(dirName,fname), encoding='utf8') as f1obj:
                    for line in f1obj:
#                         rfobj.write(line.strip().replace('+', ''))
#                         rfobj.write('\t')
                        rfobj.write(line)
                with codecs.open(os.path.join(dirName,fname.replace('train','dev')), encoding='utf8') as f1obj:
#                     # lev_devfold_04.trg
                    for line in f1obj:
#                         rfobj.write(line.strip().replace('+', ''))
#                         rfobj.write('\t')
                        rfobj.write(line)

Found directory: final_splits_all_data/joint/joint
magh_5.train.trg
magh_3.train.trg
magh_1.train.trg
magh_4.train.trg
magh_2.train.trg
Found directory: final_splits_all_data/joint/joint/.ipynb_checkpoints
